In [3]:
import numpy as np

import pynwb
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries
from pynwb.ecephys import ElectricalSeries, LFP

In [13]:
import fmEphys as fme

Loading DLC 2.3.0...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [14]:
ltdk = fme.read_group_h5(r'C:\Users\dmartins\Documents\Dropbox\Research\Niell_lab\Data\Gaze_shift_data\Mouse\complete_dataset\ltdk_021423_figupdate.h5')

In [15]:
ltdk['session'].unique()

array(['020322_J577TT_control_Rig2', '020522_J577RT_control_Rig2',
       '100821_J559TT_control_Rig2', '101621_J559NC_control_Rig2',
       '102721_J558NC_control_Rig2', '110421_J558LT_control_Rig2',
       '110521_J569LT_control_Rig2'], dtype=object)

In [ ]:
# M, M, M, M, F, F, M

In [16]:
row = ltdk[ltdk['session']=='020322_J577TT_control_Rig2'].iloc[0].copy()

In [ ]:
row['']

In [17]:
row['t0']

41336.927116

In [20]:
from datetime import datetime

In [22]:
row['t0']

datetime.date(2023, 2, 20)

In [27]:
row['t0']

41336.927116

In [26]:
datetime.time(second=row['t0'])

TypeError: descriptor 'time' of 'datetime.datetime' object needs an argument

In [25]:
datetime.strptime('020322', '%m%d%y') + 

TypeError: strptime() argument 1 must be str, not numpy.float64

In [ ]:
nwbfile = NWBFile(
    identifier ='020322_J577TT_HfWn',
    session_start_time = row['t0'],
    file_create_date = datetime.today().date()
    
    session_description="my first synthetic recording",
    session_start_time=datetime.now(tzlocal()),
    experimenter="",
    session_start_time=""
    file_create_date=
    lab="Niell lab",
    institution="University of Oregon",
    experiment_description="I went on an adventure with thirteen dwarves",
    session_id="LONELYMTN",
    related_publications=
    stimulus_notes=
    subject = pynwb.file.Subject(
            subject_id='J577TT',
            sex='M',
            species='Mouse'
        )
    units=
    lab_meta_data=-
)

In [ ]:
nwbfile.subject = Subject(
    subject_id="001",
    age="P90D",
    description="mouse 5",
    species="Mus musculus",
    sex="M",
)

In [ ]:
device = nwbfile.create_device(
    name="array", description="the best array", manufacturer="Probe Company 9000"
)

In [ ]:
nwbfile.add_electrode_column(name="label", description="label of electrode")

nshanks = 4
nchannels_per_shank = 3
electrode_counter = 0

for ishank in range(nshanks):
    # create an electrode group for this shank
    electrode_group = nwbfile.create_electrode_group(
        name="shank{}".format(ishank),
        description="electrode group for shank {}".format(ishank),
        device=device,
        location="brain area",
    )
    # add electrodes to the electrode table
    for ielec in range(nchannels_per_shank):
        nwbfile.add_electrode(
            group=electrode_group,
            label="shank{}elec{}".format(ishank, ielec),
            location="brain area",
        )
        electrode_counter += 1

In [ ]:
nwbfile.electrodes.to_dataframe()

In [ ]:
all_table_region = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="all electrodes",
)

In [ ]:
raw_data = np.random.randn(50, 4)
raw_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=raw_data,
    electrodes=all_table_region,
    starting_time=0.0,  # timestamp of the first sample in seconds relative to the session start time
    rate=20000.0,  # in Hz
)

In [ ]:
nwbfile.add_acquisition(raw_electrical_series)

In [ ]:
lfp_data = np.random.randn(50, 4)
lfp_electrical_series = ElectricalSeries(
    name="ElectricalSeries",
    data=lfp_data,
    electrodes=all_table_region,
    starting_time=0.0,
    rate=200.0,
)

In [ ]:
lfp = LFP(electrical_series=lfp_electrical_series)

In [ ]:
ecephys_module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
ecephys_module.add(lfp)

In [ ]:
nwbfile.add_unit_column(name="quality", description="sorting quality")

poisson_lambda = 20
firing_rate = 20
n_units = 10
for n_units_per_shank in range(n_units):
    n_spikes = np.random.poisson(lam=poisson_lambda)
    spike_times = np.round(
        np.cumsum(np.random.exponential(1 / firing_rate, n_spikes)), 5
    )
    nwbfile.add_unit(
        spike_times=spike_times, quality="good", waveform_mean=[1.0, 2.0, 3.0, 4.0, 5.0]
    )

nwbfile.units.to_dataframe()

In [ ]:
with NWBHDF5IO("ecephys_tutorial.nwb", "w") as io:
    io.write(nwbfile)

## behavior data

In [ ]:
timestamps = np.linspace(0, 50) / 200

position_spatial_series = SpatialSeries(
    name="SpatialSeries",
    description="Position (x, y) in an open field.",
    data=position_data,
    timestamps=timestamps,
    reference_frame="(0,0) is bottom left corner",
)

position_spatial_series

In [ ]:
speed_time_series = TimeSeries(
    name="speed",
    data=speed_data,
    timestamps=timestamps,
    description="The speed of the subject measured over time.",
    unit="m/s",
)

behavioral_time_series = BehavioralTimeSeries(
    time_series=speed_time_series,
    name="BehavioralTimeSeries",
)

behavior_module.add(behavioral_time_series)

In [ ]:
reward_amount = [1., 1.5, 1., 1.5]
events_timestamps = [1., 2., 5., 6.]

time_series = TimeSeries(
    name="lever_presses",
    data=reward_amount,
    timestamps=events_timestamps,
    description="The water amount the subject received as a reward.",
    unit="ml",
)

behavioral_events = BehavioralEvents(time_series=time_series, name="BehavioralEvents")

behavior_module.add(behavioral_events)

In [ ]:
run_intervals = IntervalSeries(
    name="running",
    description="Intervals when the animal was running.",
    data=[1, -1, 1, -1, 1, -1],
    timestamps=[0.5, 1.5, 3.5, 4.0, 7.0, 7.3],
)

behavioral_epochs = BehavioralEpochs(name="BehavioralEpochs")

behavioral_epochs.add_interval_series(run_intervals)

In [ ]:
pupil_diameter = TimeSeries(
    name="pupil_diameter",
    description="Pupil diameter extracted from the video of the right eye.",
    data=np.linspace(0.001, 0.002, 50),
    timestamps=timestamps,
    unit="meters",
)

pupil_tracking = PupilTracking(time_series=pupil_diameter, name="PupilTracking")

behavior_module.add(pupil_tracking)

In [ ]:
right_eye_position = np.linspace(-20, 30, 50)

right_eye_positions = SpatialSeries(
    name="right_eye_position",
    description="The position of the right eye measured in degrees.",
    data=right_eye_position,
    timestamps=timestamps,
    reference_frame="bottom left",
    unit="degrees",
)

eye_tracking = EyeTracking(name="EyeTracking", spatial_series=right_eye_positions)

In [ ]:
image_data = np.random.randint(low=0, high=255, size=(200, 50, 50, 3), dtype=np.uint8)
behavior_images = ImageSeries(
    name="ImageSeries",
    data=image_data,
    description="Image data of an animal moving in environment.",
    unit="n.a.",
    format="raw",
    rate=1.0,
    starting_time=0.0,
)

nwbfile.add_acquisition(behavior_images)